# NBA Game Outcome

In [1]:
# Import Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
games_df = pd.concat(map
                     (pd.read_csv, 
                          ['data/game_details/nba_game_detailsSeason=2021-22.csv', 
                           'data/game_details/nba_game_detailsSeason=2020-21.csv', 
                           'data/game_details/nba_game_detailsSeason=2019-20.csv',
                           'data/game_details/nba_game_detailsSeason=2018-19.csv',
                           'data/game_details/nba_game_detailsSeason=2017-18.csv',
                           'data/game_details/nba_game_detailsSeason=2016-17.csv',
                           'data/game_details/nba_game_detailsSeason=2015-16.csv',
                           'data/game_details/nba_game_detailsSeason=2014-15.csv',
                           'data/game_details/nba_game_detailsSeason=2013-14.csv',
                           'data/game_details/nba_game_detailsSeason=2012-13.csv']), ignore_index=True)

In [17]:
y = []
for i in range(0,3000):#len(games_df)):
# Check Matchup to see which team is Home
    if "@" in games_df["MATCHUP"][i]:
        away_team = games_df["TEAM_ABBREVIATION"]
        home_team = games_df["MATCHUP"][i][-3:]
        
        #Adding win/loss to home team
        if games_df["PLUS_MINUS"][i] > 0:
            y.append(0)
        else:
            y.append(1)
         
    else:
        home_team = games_df["TEAM_ABBREVIATION"]
        away_team = games_df["MATCHUP"][i][-3:]
        
        #Adding win/loss to home team
        if games_df["PLUS_MINUS"][i] > 0:
            y.append(1)
        else:
            y.append(0)
last_10h = games_df[games_df["TEAM_ABBREVIATION"] == home_team][games_df["GAME_DATE"]< games_df["GAME_DATE"][i]].head(10)
last_10a = games_df[games_df["TEAM_ABBREVIATION"] == away_team][games_df["GAME_DATE"]< games_df["GAME_DATE"][i]].head(10)


C:\Users\donjo\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [18]:
(last_10h[last_10h["WL"]=="W"])

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
3027,22020,1610612742,DAL,Dallas Mavericks,22000673,2021-03-24,DAL @ MIN,W,240,48,...,39,53,27,5,6,10,22,128,20,1
3075,22020,1610612742,DAL,Dallas Mavericks,22000652,2021-03-21,DAL @ POR,W,240,48,...,44,52,23,4,3,11,17,132,40,1
3140,22020,1610612742,DAL,Dallas Mavericks,22000625,2021-03-17,DAL vs. LAC,W,240,40,...,37,43,14,7,7,18,16,105,16,1
3211,22020,1610612742,DAL,Dallas Mavericks,22000592,2021-03-13,DAL @ DEN,W,240,42,...,37,48,26,5,3,8,19,116,13,1
3257,22020,1610612742,DAL,Dallas Mavericks,22000567,2021-03-10,DAL vs. SAS,W,240,43,...,38,51,29,3,1,15,19,115,11,1
3285,22020,1610612742,DAL,Dallas Mavericks,22000549,2021-03-03,DAL vs. OKC,W,240,32,...,46,53,18,8,3,15,13,87,9,1
3314,22020,1610612742,DAL,Dallas Mavericks,22000529,2021-03-01,DAL @ ORL,W,240,47,...,39,46,29,6,6,8,12,130,6,1


In [20]:
# Creating rows for dataframe
trans_df = pd.DataFrame()
y = []
for i in range(0,3000):#len(games_df)):
    # Check Matchup to see which team is Home
    if "@" in games_df["MATCHUP"][i]:
        away_team = games_df["TEAM_ABBREVIATION"]
        home_team = games_df["MATCHUP"][i][-3:]
        
        #Adding win/loss to home team
        if games_df["PLUS_MINUS"][i] > 0:
            y.append(0)
        else:
            y.append(1)
         
    else:
        home_team = games_df["TEAM_ABBREVIATION"]
        away_team = games_df["MATCHUP"][i][-3:]
        
        #Adding win/loss to home team
        if games_df["PLUS_MINUS"][i] > 0:
            y.append(1)
        else:
            y.append(0)
    # row list for df
    row = []
    
    #Last 10 games df for both teams
    last_10h = games_df[games_df["TEAM_ABBREVIATION"] == home_team][games_df["GAME_DATE"]< games_df["GAME_DATE"][i]].head(10)
    last_10a = games_df[games_df["TEAM_ABBREVIATION"] == away_team][games_df["GAME_DATE"]< games_df["GAME_DATE"][i]].head(10)
    
    win_rateH = 0
    win_rateA = 0
    
    needed_features = ["FGA", "FG_PCT", "FG3_PCT", "DREB", "REB", "AST"]
    
    for feature in needed_features:
        row.append(round((sum(last_10h[feature])/10),2))
        row.append(round((sum(last_10a[feature])/10),2))
    
    row.append(round((len(last_10h[last_10h["WL"]=="W"])/10),2))
    row.append(round((len(last_10a[last_10a["WL"]=="W"])/10),2))
    
    trans_df = trans_df.append(pd.DataFrame([row]), ignore_index=True)

trans_df=trans_df.set_axis(["FGA_HOME","FGA_AWAY", "FG_PCT_HOME", "FG_PCT_AWAY", "FG3_PCT_HOME", "FG3_PCT_AWAY", 
                               "DREB_HOME", "DREB_AWAY", "REB_HOME", "REB_AWAY", "AST_HOME", "AST_AWAY", "WIN_RATE_HOME", "WIN_RATE_AWAY"], axis=1)
        
trans_df["WIN"] = y

trans_df.head()

C:\Users\donjo\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\donjo\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,FGA_HOME,FGA_AWAY,FG_PCT_HOME,FG_PCT_AWAY,FG3_PCT_HOME,FG3_PCT_AWAY,DREB_HOME,DREB_AWAY,REB_HOME,REB_AWAY,AST_HOME,AST_AWAY,WIN_RATE_HOME,WIN_RATE_AWAY,WIN
0,85.2,87.7,0.49,0.46,0.34,0.33,34.5,34.7,44.3,45.8,29.1,24.5,0.6,0.5,1
1,87.7,88.9,0.46,0.44,0.33,0.39,34.7,33.6,45.8,44.1,24.5,24.3,0.5,0.4,1
2,87.7,87.1,0.46,0.48,0.33,0.34,34.7,31.3,45.8,39.3,24.5,23.7,0.5,0.3,1
3,91.5,87.7,0.48,0.46,0.41,0.33,37.7,34.7,49.4,45.8,25.3,24.5,0.8,0.5,1
4,86.8,87.7,0.44,0.46,0.38,0.33,36.2,34.7,48.6,45.8,22.0,24.5,0.5,0.5,0


# Read the Data and Perform Basic Data Cleaning

In [4]:
columns = ["HOME_TEAM_ID", "VISITOR_TEAM_ID", "SEASON", "PTS_home", "FG_PCT_home", "FT_PCT_home", "FG3_PCT_home", "AST_home", "REB_home", "PTS_away", "FG_PCT_away", "FT_PCT_away", "FG3_PCT_away", "AST_away", "REB_away"]

target = ["HOME_TEAM_WINS"]

In [31]:
"""# Load the data from AWS - REVIEW WITH TEAM
file_path ="https://YOUR-BUCKET-NAME.s3.amazonaws.com/INSERTLINK"
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null comumns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Convert the target column values to win/loss based on their values
x = {'1': 'Win'}   
df = df.replace(x)

x = {'0': 'Loss'}   
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

#create column with FGA difference between teams
trans_df["FGA_DIFF"]=trans_df["FGA_HOME"]-trans_df["FGA_AWAY"]
trans_df.drop(columns=["FGA_HOME","FGA_AWAY"], inplace=True)
trans_df.head()

#Create column with FG percent ratio between teams
trans_df["FG_PCT_RATIO"]=trans_df["FG_PCT_HOME"]/trans_df["FG_PCT_AWAY"]
trans_df.drop(columns=["FG_PCT_HOME", "FG_PCT_AWAY"], inplace=True)

#Create column with 3-point FG percent ratio between teams
trans_df["FG3_PCT_RATIO"] = trans_df["FG3_PCT_HOME"]/trans_df["FG3_PCT_AWAY"]
trans_df.drop(columns=["FG3_PCT_HOME", "FG3_PCT_AWAY"], inplace = True)

#Create column with Defensive rebound difference between teams
trans_df["DREB_DIFF"]=trans_df["DREB_HOME"]-trans_df["DREB_AWAY"]
trans_df.drop(columns=["DREB_HOME", "DREB_AWAY"], inplace =True)

#Create column with rebound difference between teams
trans_df["REB_DIFF"]=trans_df["REB_HOME"]-trans_df["REB_AWAY"]
trans_df.drop(columns=["REB_HOME","REB_AWAY"], inplace=True)

#Create column with assist difference between teams
trans_df["AST_DIFF"]=trans_df["AST_HOME"]-trans_df["AST_AWAY"]
trans_df.drop(columns=["AST_HOME","AST_AWAY"], inplace=True)

win=trans_df["WIN"]
trans_df.head()

win=trans_df["WIN"]
trans_df.drop(columns=["WIN"], inplace=True)
trans_df["WIN"]=win
trans_df.head()
"""

,FGA_DIFF,FG_PCT_RATIO,FG3_PCT_RATIO,DREB_DIFF,REB_DIFF,AST_DIFF,WIN
0,-0.60,1.042553,0.971429,3.25,3.30,2.95,1
1,-1.00,1.021739,0.897436,-0.80,-0.70,0.85,1
2,-0.45,0.959184,0.972222,0.25,1.70,2.35,1
3,2.85,1.042553,1.114286,2.10,2.50,-0.50,1
4,1.70,0.914894,1.000000,1.75,4.95,-3.60,0


# Split the Data into Training and Testing

In [21]:
#Creating input and output data
trans_df["WIN"] = y
y = trans_df["WIN"]
X_df = trans_df.drop(["WIN"], axis=1)

In [22]:
# Check the balance of our target values
y.value_counts()

1    1646
0    1354
Name: WIN, dtype: int64

In [23]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_df, y, random_state=1, stratify=y)

# Balanced Random Forest Classifier

In [24]:
# Resample the training data with the BalancedRandomForestClassifier
random_forest = BalancedRandomForestClassifier(n_estimators = 100)
random_forest = random_forest.fit(X_train, y_train)

In [25]:
# Calculated the balanced accuracy score
y_pred = random_forest.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.57365571321882

In [26]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[195, 143],
       [177, 235]], dtype=int64)

In [27]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.52      0.58      0.57      0.55      0.57      0.33       338
          1       0.62      0.57      0.58      0.59      0.57      0.33       412

avg / total       0.58      0.57      0.57      0.57      0.57      0.33       750



In [28]:
# List the features sorted in descending order by feature importance
feature_names = X_df.columns
sorted(zip(random_forest.feature_importances_, feature_names), reverse=True)

[(0.09134130089781149, 'FGA_AWAY'),
 (0.09037506425659102, 'DREB_HOME'),
 (0.08911613275155972, 'AST_HOME'),
 (0.08842462557313013, 'AST_AWAY'),
 (0.08744192734828637, 'FGA_HOME'),
 (0.08724618696288106, 'REB_HOME'),
 (0.08608556242895325, 'DREB_AWAY'),
 (0.08498448718497478, 'REB_AWAY'),
 (0.05671113517312689, 'FG3_PCT_AWAY'),
 (0.05655335956704967, 'FG3_PCT_HOME'),
 (0.04910415520375588, 'FG_PCT_AWAY'),
 (0.04842317743837433, 'FG_PCT_HOME'),
 (0.0446318125741615, 'WIN_RATE_HOME'),
 (0.039561072639343935, 'WIN_RATE_AWAY')]

In [29]:
X_train

,FGA_HOME,FGA_AWAY,FG_PCT_HOME,FG_PCT_AWAY,FG3_PCT_HOME,FG3_PCT_AWAY,DREB_HOME,DREB_AWAY,REB_HOME,REB_AWAY,AST_HOME,AST_AWAY,WIN_RATE_HOME,WIN_RATE_AWAY
398,87.8,90.7,0.50,0.46,0.36,0.33,34.0,35.2,42.9,45.6,25.7,25.0,0.6,0.5
2721,90.3,87.5,0.47,0.48,0.34,0.34,32.5,32.3,43.2,40.0,26.2,24.1,0.5,0.4
2465,92.7,89.8,0.50,0.47,0.38,0.34,37.7,34.9,48.2,45.2,25.8,25.8,0.8,0.5
2593,90.0,82.1,0.48,0.48,0.35,0.38,32.9,30.4,44.0,38.1,24.7,28.7,0.5,0.6
1355,86.9,87.2,0.47,0.44,0.39,0.35,30.6,35.4,37.9,45.1,27.4,24.0,0.4,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428,89.8,91.0,0.46,0.45,0.34,0.36,34.3,36.0,44.6,47.2,23.8,24.4,0.2,0.5
1015,89.0,83.9,0.42,0.46,0.33,0.34,35.3,33.5,46.7,41.7,23.0,26.5,0.5,0.6
2578,90.0,85.7,0.48,0.47,0.35,0.37,32.9,31.9,44.0,42.4,24.7,24.4,0.5,0.4
556,86.2,89.2,0.44,0.46,0.32,0.37,34.5,34.9,44.4,44.4,23.0,27.3,0.2,0.5


In [30]:
X_train.head(1)

,FGA_HOME,FGA_AWAY,FG_PCT_HOME,FG_PCT_AWAY,FG3_PCT_HOME,FG3_PCT_AWAY,DREB_HOME,DREB_AWAY,REB_HOME,REB_AWAY,AST_HOME,AST_AWAY,WIN_RATE_HOME,WIN_RATE_AWAY
398,87.8,90.7,0.5,0.46,0.36,0.33,34.0,35.2,42.9,45.6,25.7,25.0,0.6,0.5


In [31]:
random_forest.predict_proba(X_train.head(5))

array([[0.78, 0.22],
       [0.18, 0.82],
       [0.09, 0.91],
       [0.77, 0.23],
       [0.18, 0.82]])